### Introduction to Machine Learning
#### Voice-Controlled Wheelchair Project
##### Eliana Ferreira


In [ ]:
#2.2
import os

dataset_path = 'dataset'

classes = ['forward', 'backward', 'left', 'right', 'stop', '_silence_', '_unknown_']

# iniciar um dicionário com as diferentes classes 
classe_ficheiros = {}

for class_name in classes: 
    class_folder = os.path.join(dataset_path, class_name) # ir buscar o path completo de cada classe
    ficheiros = os.listdir(class_folder) # vai buscar todos os ficheiros daquela pasta
    classe_ficheiros[class_name] = ficheiros # atribuir os ficheiros à respetiva classe



Resposta à pergunta 2.3 

O desequilíbrio nas classes pode gerar uma série de desafios. Classes maioritárias, como "Left", "Right" e "Stop", tendem a dominar as análises estatísticas, enquanto classes minoritárias, como "Silence" e "Unknown", podem não conter exemplos suficientes para uma extração de características adequada ou para a deteção de outliers. Durante a normalização e transformação de dados, técnicas como PCA e métodos de seleção de características, como Fisher Score ou ReliefF, podem priorizar características associadas às classes dominantes, uma vez que estas contribuem mais para a variância global, ignorando informações cruciais para classes menos representadas. Esse desequilíbrio também afeta a divisão do conjunto de dados em treino, validação e teste, onde divisões aleatórias podem resultar em representações insuficientes das classes minoritárias, levando a generalizações fracas e métricas de desempenho distorcidas, como uma precisão aumentada que não reflete a capacidade real do modelo reconhecer todas as classes. No caso de classificadores como o KNN, o viés para as classes dominantes torna-se evidente, pois os vizinhos mais próximos de um ponto de consulta geralmente pertencem às classes maioritárias. Isso é exacerbado por valores grandes de k, que amplificam ainda mais o peso dessas classes. 

In [ ]:
#2.4
from scipy.io import wavfile
import numpy as np

def load_and_normalize(ficheiro, class_folder):
    ficheiro_path = os.path.join(class_folder, ficheiro) # cria um path para o ficheiro
    fs, data = wavfile.read(ficheiro_path) # carrega o ficheiro 
    data = data / np.iinfo(data.dtype).max  # normaliza o som
    return fs, data 

In [ ]:
#2.5
import os 

def envelope(sinal, window_size):
    abs_signal = np.abs(sinal) # meter todos os valores do sinal em absoluto
    half_window = window_size // 2 
    padded_signal = np.pad(abs_signal, (half_window, half_window), mode='constant', constant_values=0) # padding de zero no inicio e fim para que hajam valores para a moving window, o número de zeros é igual ao número da halfwindow
    envelope_signal = np.zeros_like(sinal)  # criar uma array com as mesmas caracteristicas que o sinal, cheia de zeros
    for i in range(len(sinal)):
        envelope_signal[i] = np.mean(padded_signal[i:i + window_size]) # cria subarrays que começam em i e terminam em i + window para criar a window       
    return envelope_signal

# exemplo só para visualizar 
# carregar o primeiro ficheiro de forward
fs, data = load_and_normalize('0a2b400e_nohash_0.wav', 'dataset/forward')

# atribuir valores a window (ímpar - 101) e a sinal (data)
window_size = 35
env = envelope(data, window_size)

# Visualizar
import matplotlib.pyplot as plt

time = np.arange(len(data)) / fs #dá-nos o tempo de cada sinal
plt.figure(figsize=(10, 5))
plt.plot(time, data, label='Sinal Original')
plt.plot(time, env, label='Envelope', color='red', linewidth=2)
plt.xlabel('Time (s)')
plt.ylabel('Amplitude')
plt.title('Sinal e Envelope')
plt.legend()
plt.show()



In [ ]:
# 2.6

import random

def visualizar_samples(classe_ficheiros, dataset_path, fs):
    classes = list(classe_ficheiros.keys())
    plt.figure(figsize=(12, 10))

    for i, class_name in enumerate(classes): # o enumerar resulta no índice e no elemento
        
        selected_file = random.choice(classe_ficheiros[class_name]) # selecionar um ficheiro random da classe dada
        class_folder = os.path.join(dataset_path, class_name) 
        _, data = load_and_normalize(selected_file, class_folder) # wavfile.read - devolve um tuple - fs,data - porque queremos só a segunda informação metemos só o placeholder
        time = np.arange(len(data)) / fs  # /fs converte os indices em tempo(de cada sample), uma vez ~que fs é a frequencia do sinal (nº de samples por segundo)
        duration_limit = min(len(data), fs) # ou 1 segundo (fs) ou então o comprimento do sinal, usamos min para se o sinal for menor que um segundo, ficar essa duração
        data = data[:duration_limit]
        time = time[:duration_limit] # para fazer plot - time e data - têm o mesmo número de elementos - se não o python levantava um ValueError porque a função plot assume que cada coordenada x tem de ter uma correspondente coordenada y

        plt.subplot(7,1, i + 1)  # 4 linhas, 2 colunas, sequencial
        plt.plot(time, data)
        plt.title(class_name.capitalize())
        plt.xlabel('Time [s]')
        plt.ylabel('Amplitude')
        plt.xlim(0, 1)
        plt.ylim(-1, 1)
        plt.subplots_adjust(hspace=1.7)

plt.show()

visualizar_samples(classe_ficheiros, dataset_path, fs)



In [ ]:
#3.1

def extract_feature1(data, envelope_signal, fs, threshold=0.035):
    start_idx = np.argmax(envelope_signal > threshold) # argmax devolve o primeiro valor que se adequa à condição
    end_idx = len(envelope_signal) - np.argmax(envelope_signal[::-1] > threshold) - 1 # procura o primeiro indice a passar o threshold na array invertida (ultimo na original
    # depois para voltar ao indice na original, subtrai o indice encontrado ao length e subtrai um porque começa em 0
    if np.max(envelope_signal) <= threshold:  # se nunca passar o threshold o incio é zero e o fim é o length - 1
        start_idx = 0
        end_idx = len(data) - 1

    duration = (end_idx - start_idx + 1) / fs # divide-se pela frequencia para ter o tempo porque T=1/fs 
    return duration #F1

#exemplo 
fs, data = load_and_normalize('0b7ee1a0_nohash_2.wav', 'dataset/forward')
env= envelope(data, window_size)
dur = extract_feature1(data, env, fs, threshold=0.035)
print(dur)



In [ ]:
#3.2 

def extract_feature2_e_feature3(envelope_signal, start_idx, end_idx):
    envelope_segment = envelope_signal[start_idx:end_idx] # obtém o sinal correspondente à duration (F1)
    F2 = np.mean(envelope_segment) # média dos valores no intervalo definido anteriormente
    F3 = np.std(envelope_segment) # desvio padrão dos valores no intervalo definido anteriormente 

    return F2, F3

 # tem que se definir start_idx e end_idx para dar call à função



In [ ]:
#3.3

def extract_feature4(data, start_idx, end_idx):
   
    signal_segment = data[start_idx:end_idx]

    energy = np.sum(np.square(signal_segment)) # soma do quadrado das amplitudes no intervalo definido anteriormente (acima do threshold)

    return energy #F4

# tem que se definir start_idx e end_idx para dar call à função



In [ ]:
#3.4

def extract_features5to24(data, start_idx, end_idx, num_bins=20):
    signal_segment = data[start_idx:end_idx]
    bin_size = len(signal_segment) // num_bins # perceber o tamanho de cada bin dividindo o sinal acima do threshold pelo nº de bins
    energies = [] 
    for i in range(num_bins):
       
        bin_start = i * bin_size
        bin_end = (i + 1) * bin_size if i < num_bins - 1 else len(signal_segment) # (i + 1) * bin_size exceto o ultimo bin, para o ultimo é o comprimento final do segmento

        # extrair o segmento do bin atual
        bin_signal = signal_segment[bin_start:bin_end]

        # calcular a energia do bin
        bin_energy = np.sum(np.square(bin_signal))

        # juntar a energia do bin à array energies 
        energies.append(bin_energy)

    return energies

   # tem que se definir start_idx e end_idx para dar call à função
   # F5 to F24 são as energias dos 20 bins


In [ ]:
#3.5
import numpy as np


def create_feature_matrices(classe_ficheiros, dataset_path, fs, num_features=24):
    total_sounds = sum(len(ficheiros) for ficheiros in classe_ficheiros.values())
    X = np.zeros((total_sounds, num_features))  # criar uma matriz com os ficheiros no y e as features no x
    Y = np.zeros((total_sounds, 1))  # criar uma matriz com os ficheiros no y e uma coluna para depois colocar a classe 

    sound_index = 0  # para atualizar a linha atual 

    for class_label, class_name in enumerate(classe_ficheiros.keys()):  # Enumerar para atribuir um número a cada classe
        class_folder = os.path.join(dataset_path, class_name)
        
        for ficheiro in classe_ficheiros[class_name]:
        
            fs, data = load_and_normalize(ficheiro, class_folder)

            # Calcular o envelope
            window_size = 101
            envelope_signal = envelope(data, window_size)

            # Extrair features
            F1 = extract_feature1(data, envelope_signal, fs)  # Duration
            F2 = np.mean(envelope_signal)  # Mean of envelope
            F3 = np.std(envelope_signal)  # Standard deviation of envelope 
            start_idx = np.argmax(envelope_signal > 0.1)
            end_idx = len(envelope_signal) - np.argmax(envelope_signal[::-1] > 0.1) - 1
            F4 = extract_feature4(data, start_idx, end_idx)  # Energia 
            F5_to_F24 = extract_features5to24(data, start_idx, end_idx)  # Energy bins
            
            # Combinar todas as features 
            features = [F1, F2, F3, F4] + F5_to_F24
        

            # X é das features e Y é das class labels 
            X[sound_index, :] = features
            Y[sound_index, 0] = class_label  # Labels
            sound_index += 1

    return X, Y

# Example usage
X, Y = create_feature_matrices(classe_ficheiros, dataset_path, fs)


print("Feature Matrix X:")
print(X)
print("Class Labels Y:")
print(Y)



In [ ]:
#guardar as matrizes com pickle 

import pickle
import os

current_dir = os.getcwd()
filename = "matrices.pkl"
filepath = os.path.join(current_dir, filename)
with open(filepath, 'wb') as file:
    pickle.dump((X, Y), file)

with open(filepath, 'rb') as file:
    X_loaded, Y_loaded = pickle.load(file)

print("X_loaded shape:", X_loaded.shape)
print("Y_loaded shape:", Y_loaded.shape)

print(Y_loaded)




In [ ]:
# 3.6
from scipy.stats import shapiro, kruskal, f_oneway
import pandas as pd
import matplotlib.pyplot as plt


def verificar_normalidade(X, Y):
    n_features = X.shape[1] #nº de features
    results = []
    classes = np.unique(Y) # agrupar as classes
    
    for feature in range(n_features):
        feature_name = f"Feature {feature + 1}"
        feature_results = {
            "Feature Name": feature_name,
            "Normality p-value": None,
            "Test Applied": None,
            "Test p-value": None
        }

        for classe in classes:
            # seleciona os sons da feature atual de X que têm a classe atual em Y atribuída # flatten - torna array 
            feature_values = X[Y.flatten() == classe, feature]

            _, p_value_normalidade = shapiro(feature_values) # porque shapiro devolve dois valores
            feature_results["Normality p-value"] = p_value_normalidade

            if p_value_normalidade < 0.05:
                feature_results["Test Applied"] = "Kruskal-Wallis"
                stat, test_p_value = kruskal(*feature_values)
                feature_results["Test p-value"] = test_p_value
            else:
                feature_results["Test Applied"] = "ANOVA"
                _, anova_p_value = f_oneway(*feature_values)
                feature_results["Test p-value"] = anova_p_value

            results.append(feature_results)

        results_df = pd.DataFrame(results)
        results_df = results_df.sort_values(by="Test p-value", ascending=True)

    print("\nRanked Features by Statistical Significance:")
    print(results_df)
    
    return results_df

verificar_normalidade(X, Y)

#para vermos que a distribuição não é normal
def plot_example(X, Y, feature_index=6, class_value=5):

    feature_values = X[Y.flatten() == class_value, feature_index]

    # criar histograma
    plt.figure(figsize=(10, 6))
    plt.hist(feature_values, bins=20, edgecolor='black')
    plt.title(f'Distribuição da Classe {class_value} na Feature {feature_index + 1} ')
    plt.xlabel('Value')
    plt.ylabel('Frequency')
    plt.grid(axis='y', alpha=0.75)
    plt.show()

plot_example(X, Y)



3.6.1 

O teste de Kruskal-Wallis H é o teste estatístico mais apropriado para esta situação, uma vez que o objetivo principal é avaliar se uma determinada feature pode discriminar significativamente entre diferentes classes, dado que os dados não seguem uma distribuição normal. Ao contrário dos testes paramétricos, como a ANOVA, que exigem a suposição de normalidade, o teste de Kruskal-Wallis é um método não paramétrico que compara as distribuições da feature entre várias classes, sem qualquer suposição de normalidade. Se houvesse apenas duas classes, poderia ser utilizado o teste de Mann-Whitney U, mas existem 7 classes.
Ao classificar os dados e comparar as classificações entre as classes, o teste de Kruskal-Wallis avalia efetivamente se as distribuições da feature diferem significativamente entre os grupos, proporcionando assim um método robusto para identificar as features que podem distinguir entre as classes. A capacidade do teste de lidar com dados não normais enquanto compara múltiplos grupos torna-o uma escolha ideal para garantir que o poder discriminatório da feature seja adequadamente avaliado em dados do mundo real, onde a normalidade nem sempre pode ser garantida.

In [ ]:
#4.1 
#univariada por classe
def detect_outliers(X, Y, feature_index, method='iqr', iqr_multiplier=1.5, z_threshold=3.0):
  
    classes = np.unique(Y)  
    feature_values = X[:, feature_index]  
    outliers_per_class = {}  

    plt.figure(figsize=(10, 6))  

    for i, classe in enumerate(classes): # filtrar os valores de cada classe
        class_indexes = np.where(Y.flatten() == classe)[0]
        class_values = feature_values[class_indexes]

        if method == 'iqr':
            Q1 = np.percentile(class_values, 25)  
            Q3 = np.percentile(class_values, 75)  
            IQR = Q3 - Q1
            lower_bound = Q1 - (iqr_multiplier * IQR)
            upper_bound = Q3 + (iqr_multiplier * IQR)
            outliers_indexes = np.where((class_values < lower_bound) | (class_values > upper_bound))[0]

        elif method == 'zscore':
            mean = np.mean(class_values)
            std = np.std(class_values)
            z_scores = (class_values - mean) / std
            outliers_indexes = np.where(np.abs(z_scores) > z_threshold)[0]

        else:
            raise ValueError("Invalid method. Use 'iqr' or 'zscore'.")

        # Cir buscar os indices especificos das classes e passá-los para indices globais
        global_outliers_indexes = class_indexes[outliers_indexes]

        # densidade
        no = len(global_outliers_indexes)  # nº de outliers
        nr = len(class_values)  # nº total de valores numa classe
        outliers_density = (no / nr) * 100

        outliers_per_class[classe] = {
            "outliers_density": outliers_density,
            "outliers_indexes": global_outliers_indexes.tolist()
        }

        print(f"Class {classe}:")
        print(f"  Outliers Density: {outliers_density:.2f}%")
        print(len(outliers_indexes))

    
        class_inliers = np.setdiff1d(class_indexes, global_outliers_indexes)
        plt.scatter(
            np.full_like(class_inliers, i, dtype=int),
            feature_values[class_inliers],
            color="blue",
            alpha=0.6,
            label=f"Class {classe} (Inliers)" if i == 0 else "",
        )
        plt.scatter(
            np.full_like(global_outliers_indexes, i, dtype=int),
            feature_values[global_outliers_indexes],
            color="red",
            alpha=0.6,
            label=f"Class {classe} (Outliers)" if i == 0 else "",
            edgecolors="black",
        )

    # Plot details
    plt.title(f"Feature {feature_index + 1} ({method.upper()} Method)")
    plt.xlabel("Class")
    plt.ylabel("Feature Value")
    plt.xticks(range(len(classes)), classes)  # Display class labels on x-axis
    plt.legend()
    plt.grid(True)
    plt.show()

    return outliers_per_class

detect_outliers(X, Y, 1, method='zscore', z_threshold=3.0)
detect_outliers(X, Y, 1, method='iqr', iqr_multiplier=1.5)





Resposta à pergunta 4.2.3

A densidade de outliers calculada pelo método IQR tende a ser superior àquela obtida pelo método z-score devido às diferenças nas abordagens de detecção de outliers. O método IQR identifica outliers com base na amplitude interquartil (a diferença entre o 75º e o 25º percentil) e, com um multiplicador de 1.5, qualquer valor fora do intervalo determinado por essa amplitude é considerado um outlier. Esse critério tende a ser mais sensível a valores extremos, resultando numa maior densidade de outliers, especialmente quando a distribuição dos dados tem caudas mais longas ou dispersão considerável. Por outro lado, o z-score calcula a distância padrão de cada ponto em relação à média. Valores com um z-score superior a 3, por exemplo, são considerados outliers. Quanto maior o valor do z-score (como 3.5 ou 4), maior a probabilidade de um ponto ser classificado como outlier, pois a margem para a variação em torno da média é reduzida, tornando a detecção de outliers mais rigorosa. Isso explica o aumento da densidade de outliers quando o z-threshold é aumentado, já que menos valores ficam dentro do intervalo de confiança definido pelo z-score, resultando em mais pontos sendo considerados outliers. Portanto, enquanto o IQR é mais sensível a dispersões extremas, o z-score tende a ser mais conservador, e ao aumentar o valor do z-threshold, a densidade de outliers diminui, já que a definição de outlier se torna mais restritiva.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pickle

#criar histogramas para cada classe

def plot_histograms_per_class(X_loaded, Y_loaded):
    classes = np.unique(Y_loaded)  
    feature_1 = X_loaded[:, 0]  
    feature_2 = X_loaded[:, 1]
    print(feature_1)  

    # Feature 1
    for classe in classes:
        values = feature_1[Y_loaded.flatten() == classe]  
        plt.figure(figsize=(8, 5))
        plt.hist(values, bins=20, color='pink', edgecolor='black')
        plt.title(f"Histograma da Feature 1 para a Classe {classe}")
        plt.xlabel("Value")
        plt.ylabel("Frequency")
        plt.grid(axis="y")
        plt.show()

    # Feature 2 
    for classe in classes:
        values = feature_2[Y_loaded.flatten() == classe]  # Filter Feature 2 values by class
        plt.figure(figsize=(8, 5))
        plt.hist(values, bins=20, edgecolor='black')
        plt.title(f"Histograma da Feature 2 para a Classe {classe}")
        plt.xlabel("Value")
        plt.ylabel("Frequency")
        plt.grid(axis="y")
        plt.show()

# Fazer histogramas F1 e F2
plot_histograms_per_class(X_loaded, Y_loaded)


In [ ]:
#4.3 
from sklearn.cluster import KMeans

F1 = X[:, 0]
F2 = X[:, 1]

def kmeans_clustering(F1, F2, num_clusters):

    features = np.column_stack((F1, F2)) # juntas as features numa matriz 2D

    kmeans = KMeans(n_clusters=num_clusters, random_state=42)
    labels = kmeans.fit_predict(features)
    centroids = kmeans.cluster_centers_

    # Calcula as distâncias dos pontos ao centróide do cluster ao qual pertencem
    distances = np.linalg.norm(features - centroids[labels], axis=1)

    # Define um limiar com base no percentil
    outliers_indexes = []
    for i in range(num_clusters):
        cluster_idxs = np.where(labels==i)[0]
        
        #de indices dos cluster a indices dos outliers no cluster para indices globais
        distance_threshold = np.percentile(distances[cluster_idxs], 99.95)
        cluster_outliers_indexes = np.where(distances[cluster_idxs] > distance_threshold)[0]
        outliers_indexes.extend(cluster_idxs[cluster_outliers_indexes])
    
    # Identifica os índices dos outliers
    outliers = features[outliers_indexes]
    print(len(outliers))
    
    return outliers_indexes, labels, features, centroids, outliers

outliers_indexes, labels, features, centroids, outliers = kmeans_clustering(F1, F2, num_clusters=10)

# Plot dos resultados 
plt.figure(figsize=(12, 8))

# Plot the clusters
for cluster in np.unique(labels):
    cluster_points = features[labels == cluster]
    plt.scatter(cluster_points[:, 0], cluster_points[:, 1], label=f"Cluster {cluster}", alpha=0.6)

# Outliers a vermelho
plt.scatter(outliers[:, 0], outliers[:, 1], color="red", label="Outliers", edgecolor="black", s=50)

# Plot dos centroides
plt.scatter(centroids[:, 0], centroids[:, 1], color="black", marker="x", s=50, label="Centroids")

plt.title("KMeans Clustering with Outliers")
plt.xlabel("Feature 1 (F1)")
plt.ylabel("Feature 2 (F2)")
plt.legend()
plt.grid(alpha=0.5)
plt.show()


kmeans_clustering(F2, F1, 10)



Resposta à perguta 4.4

Ao aplicar o algoritmo K-means para identificar os outliers no conjunto de dados, com diferentes números de clusters, foi possível observar que os 14 outliers identificados são pontos que se encontram mais dispersos dentro do espaço de cada cluster, distantes dos seus centróides. Esses outliers representam amostras atípicas, cuja distância em relação aos clusters é significativamente maior, o que foi determinado com base no limiar de 99.95 das distâncias. Esse limiar é bastante restritivo, o que explica a identificação de um número reduzido de outliers. A análise com diferentes números de clusters mostrou que, ao variar a quantidade de agrupamentos, a dispersão dentro dos clusters pode aumentar ou diminuir, afetando a identificação dos outliers. A visualização gráfica dos resultados ilustra bem como os outliers se distribuem nas extremidades dos clusters.

In [ ]:
# 4.5
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN

def dbscan_clustering(values, eps, min_samples):
 
    dbscan = DBSCAN(eps=eps, min_samples=min_samples)
    labels = dbscan.fit_predict(values)
    return labels


def plot_dbscan_results(values, labels):
    plt.figure(figsize=(10, 8))
    unique_labels = set(labels)
    colors = plt.cm.Spectral(np.linspace(0, 1, len(unique_labels)))
    
    for k, col in zip(unique_labels, colors):
        if k == -1:
            col = 'k'
        class_member_mask = (labels == k)
        xy = values[class_member_mask]
        plt.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=col, markeredgecolor='k', markersize=6)
    
    plt.title('DBSCAN Clustering Results')
    plt.xlabel('Feature 1')
    plt.ylabel('Feature 2')
    plt.show()

features = np.column_stack((F1, F2))

eps = 0.5  
min_samples = 5  
labels = dbscan_clustering(features, eps, min_samples)

plot_dbscan_results(features, labels)





Resposta à pergunta 4.6

A principal diferença entre as abordagens univariada e multivariada para detecção de outliers está na forma como consideram as variáveis. A abordagem univariada, como o IQR e o Z-Score, analisa cada variável de forma isolada, o que a torna simples e eficaz em identificar outliers em dados unidimensionais ou em casos onde as variáveis não estão inter-relacionadas. No entanto, essa abordagem não captura as interações entre as variáveis, o que pode ser uma limitação significativa em dados multidimensionais, onde as variáveis podem estar correlacionadas. Já a abordagem multivariada, como o DBSCAN e o K-Means, considera simultaneamente todas as variáveis e as suas interações, permitindo identificar outliers em dados complexos e multidimensionais. O DBSCAN, por exemplo, é eficaz em identificar clusters de forma irregular e detectar outliers em regiões de baixa densidade, enquanto o K-Means, embora útil para dados bem agrupados, pode ser sensível a outliers devido ao uso de centróides. Além disso, enquanto a abordagem univariada é mais rápida e simples de aplicar, ela pode não ser capaz de lidar com estruturas de dados mais complexas, onde as interações entre as variáveis são importantes. Por outro lado, as abordagens multivariadas, embora mais computacionalmente exigentes, são mais adequadas para identificar padrões e outliers em dados onde as variáveis influenciam umas às outras. Assim, a escolha entre essas abordagens depende da natureza dos dados: se são simples e independentes, a univariada pode ser suficiente; se são complexos e interdependentes, as abordagens multivariadas são mais eficazes. Neste caso mostra-se mais eficaz a abordagem multivariada para então expressar essa relação entre as features e resultar em outliers reais.

In [ ]:
#5 

import numpy as np
from scipy import stats

def normalize_zscore(data, axis=0):
    mean = np.mean(data, axis=axis, keepdims=True)  # média de cada coluna
    std = np.std(data, axis=axis, keepdims=True)    # desvio padrao

    std[std == 0] = 1 # devolver 1 quanto a divisão +e feita por zero

    return (data - mean) / std   #devolve o z-score


def pca_with_explained_variance(data, variance_threshold=0.97):

    data = normalize_zscore(data)
 
    # centralizar 
    mean = np.mean(data, axis=0)
    centered_data = data - mean

    # calcular a matriz de covariância
    covariance_matrix = np.cov(centered_data, rowvar=False)

    #  eigenvalues e eigenvectors
    eigenvalues, eigenvectors = np.linalg.eig(covariance_matrix)

    # ordem descrescente
    sorted_indices = np.argsort(eigenvalues)[::-1]
    eigenvalues = eigenvalues[sorted_indices]
    eigenvectors = eigenvectors[:, sorted_indices]

    # variância explicada cumulativa
    total_variance = np.sum(eigenvalues)
    explained_variance_ratio = eigenvalues / total_variance
    print("Cumulative Variance (per component):")
    for i, var in enumerate(explained_variance_ratio, start=1):
        print(f"Component {i}: {var:.4f}")
    cumulative_variance = np.cumsum(explained_variance_ratio)
    
   
    # nº de componentes para a variancia explicada desejada
    n_components = np.searchsorted(cumulative_variance, variance_threshold) + 1

    # selecionar os eigenvectors correspondentes
    principal_components = eigenvectors[:, :n_components]
    print(n_components)

    # projetar esses dados nesses vetores
    transformed_data = np.dot(centered_data, principal_components)

    return transformed_data, principal_components, eigenvalues, n_components

pca_with_explained_variance(X, variance_threshold=0.90)





Resposta à pergunta 5.1.1

Ao realizar a Análise de Componentes Principais (PCA) nos dados, a importância de cada componente pode ser observada através da variância explicada por cada um dos eigenvectors (componentes principais). As componentes com maiores eigenvalues são responsáveis por explicar a maior parte da variabilidade dos dados. Nos resultados apresentados, a primeira componente explica 57.70% da variabilidade, a segunda componente contribui com 13.37%, e a terceira com 8.82%. As componentes seguintes explicam progressivamente menos variabilidade. Para explicar 90% da variabilidade dos dados, são necessárias as seis primeiras componentes, já que a soma da variância cumulativa das seis primeiras componentes atinge os 90.04%. Isso significa que, para preservar a maior parte da informação dos dados e reduzir a dimensionalidade sem perder muita variação, deveríamos utilizar as seis primeiras variáveis transformadas. A normalização dos dados com o z-score antes da aplicação da PCA foi crucial, pois garantiu que todas as variáveis tivessem a mesma escala, permitindo uma comparação justa entre elas ao calcular a variância explicada.

In [ ]:
#aplicar o fisherscore

#normalizar primeiro!

def fisher_score(X, Y):

    normalize_zscore(X)

    Y = Y.flatten() 
    unique_classes = np.unique(Y)
    n_features = X.shape[1]

    # array para os fisher scores
    fisher_scores = np.zeros(n_features)

    # média de cada feature
    overall_mean = np.mean(X, axis=0)

    for feature_idx in range(n_features):
        inter_class_var = 0
        intra_class_var = 0

        for cls in unique_classes:
            # ficheiros de cada classe
            X_cls = X[Y == cls]
            
            # média e variância dentro de cada classe
            cls_mean = np.mean(X_cls[:, feature_idx])
            cls_var = np.var(X_cls[:, feature_idx])

            # tamanho da classe
            cls_size = X_cls.shape[0]

            inter_class_var += cls_size * ((cls_mean - overall_mean[feature_idx]) ** 2)
            intra_class_var += cls_size * cls_var

        # evitar divisão por 0
        if intra_class_var != 0:
            fisher_scores[feature_idx] = inter_class_var / intra_class_var
        else:
            fisher_scores[feature_idx] = 0

    return fisher_scores

fisher_scores = fisher_score(X, Y)
top_10_fisher = np.argsort(fisher_scores)[-10:][::-1]
print("Top 10 features based on Fisher Score:", top_10_fisher)


In [ ]:
#aplicar o refiefF
from sklearn.neighbors import NearestNeighbors

def reliefF(X, Y, k=10):

    normalize_zscore(X)

    Y = Y.flatten() 
    n_samples, n_features = X.shape
    relief_scores = np.zeros(n_features)
    unique_classes = np.unique(Y)

    # knn para descobrir vizinhos mais próximos
    knn = NearestNeighbors(n_neighbors=k + 1) #porque a amostra em si também conta
    knn.fit(X)
    neighbors = knn.kneighbors(X, return_distance=False)

    for i in range(n_samples):
        target_class = Y[i]
        same_class_neighbors = neighbors[i][Y[neighbors[i]] == target_class][1:k+1]
        diff_class_neighbors = neighbors[i][Y[neighbors[i]] != target_class][:k]

        for feature_idx in range(n_features):
            # recompensar os que são da mesma classe
            if len(same_class_neighbors) > 0:
                relief_scores[feature_idx] -= np.sum(
                    np.abs(X[i, feature_idx] - X[same_class_neighbors, feature_idx])
                ) / len(same_class_neighbors)

            # penalizar os que são de classe diferente 
            if len(diff_class_neighbors) > 0:
                relief_scores[feature_idx] += np.sum(
                    np.abs(X[i, feature_idx] - X[diff_class_neighbors, feature_idx])
                ) / len(diff_class_neighbors)

    return relief_scores

relief_scores = reliefF(X, Y)
top_10_relief = np.argsort(relief_scores)[-10:][::-1]
print("Top 10 features based on Relief Score:", top_10_relief)

Resposta à pergunta 5.2.1

Top 10 features based on Fisher Score: [ 0  4  5  1  2  6  7  8 23  3 ]

Top 10 features based on Relief Score: [ 9 14 11  4 12 15  3 13 23  8]

A diferença nos top 10 features identificados pelo Fisher Score e pelo ReliefF decorre das abordagens distintas que cada método adota para avaliar a importância das características. O Fisher Score é baseado na ideia de separar as classes, avaliando a razão entre a variabilidade entre as classes (inter-classe) e a variabilidade dentro das classes (intra-classe) para cada característica. Ou seja, este favorece características que conseguem distinguir bem entre as diferentes classes. Já o ReliefF foca na diferença entre vizinhos da mesma classe e vizinhos de classes diferentes, penalizando características que tornam as classes mais semelhantes e recompensando aquelas que ajudam a distinguir as classes. Como resultado, o Fisher Score tende a selecionar características que têm uma maior separabilidade entre as classes, enquanto o ReliefF prioriza características que têm um maior poder discriminativo local entre vizinhos de classes distintas. Esse contraste explica a escolha de características diferentes entre os dois métodos, refletindo as diferentes estratégias de avaliação da importância das variáveis.

Resposta à pergunta 5.2.4

As abordagens de Feature Transformation (como o PCA) e Feature Selection (como o Fisher Score e o ReliefF) têm objetivos semelhantes, que são identificar e melhorar a representação das variáveis de entrada, mas diferem significativamente na forma como alcançam esse objetivo.

O Principal Component Analysis (PCA), utilizado na Feature Transformation, transforma as variáveis originais num novo conjunto de variáveis, chamadas de componentes principais, que são combinações lineares das variáveis originais. O principal objetivo do PCA é reduzir a dimensionalidade do conjunto de dados enquanto mantém a maior parte da variabilidade (informação) presente nos dados. Uma das vantagens do PCA é a sua capacidade de combinar as características em componentes que capturam a maior parte da variação nos dados, facilitando a visualização e análise dos dados em espaços de menor dimensão. Contudo, uma desvantagem do PCA é que as novas componentes podem ser difíceis de interpretar, pois são combinações das características originais. Além disso, o PCA é sensível a escalas diferentes das variáveis, o que torna a normalização das características uma etapa importante.

Por outro lado, as técnicas de Feature Selection (como o Fisher Score e o ReliefF) não transformam os dados, mas selecionam um subconjunto das variáveis originais que são consideradas mais relevantes para o modelo. O Fisher Score é um método estatístico que avalia a separabilidade entre as classes, escolhendo as características que maximizam a diferença entre elas. O ReliefF, por sua vez, baseia-se na ideia de distâncias entre pontos de dados próximos e pode capturar interações não-lineares, sendo mais eficaz para problemas em que as relações entre as variáveis são complexas. Uma vantagem dessas técnicas é que elas mantêm as características originais, o que pode facilitar a interpretação dos resultados. Além disso, elas ajudam a eliminar características irrelevantes ou redundantes, podendo melhorar a performance de modelos de aprendizagem computacional. No entanto, uma limitação dessas abordagens é que elas podem ser sensíveis a outliers e, dependendo do método, podem não capturar bem interações não-lineares ou relações complexas entre as variáveis.

In [ ]:
#6.1
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score

#TTSplit 

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

#TVT com TT split nested 

def nested_tvt_split(X, Y, test_size=0.3, validation_size=0.3, random_state=42):

    X_train_full, X_test, Y_train_full, Y_test = train_test_split(
        X, Y, test_size=test_size, random_state=random_state
    )

    # Inner Train-Validation split
    X_train, X_val, Y_train, Y_val = train_test_split(
        X_train_full, Y_train_full, test_size=validation_size, random_state=random_state
    )

    return X_train, X_val, X_test, Y_train, Y_val, Y_test

X_train, X_val, X_test, Y_train, Y_val, Y_test = nested_tvt_split(X, Y)


#K-Folds 

def k_fold_cross_validation(X, y, k=5, model=None):
   
    model = KNeighborsClassifier()  

    kf = KFold(n_splits=k, shuffle=True, random_state=42)  # Initialize K-Fold
    fold_accuracies = []  # To store accuracy of each fold

    for train_index, test_index in kf.split(X):
        # dividir entre teste e treino
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        # Treinar o modelo
        model.fit(X_train, y_train)

        # Predict 
        y_pred = model.predict(X_test)

        # avaliar o modelo
        accuracy = accuracy_score(y_test, y_pred)
        fold_accuracies.append(accuracy)

        print(f"Fold Accuracy: {accuracy:.2f}")

    # Calcular a accuracy média dos folds
    mean_accuracy = sum(fold_accuracies) / len(fold_accuracies)
    print(f"\nMean Accuracy across {k} folds: {mean_accuracy:.2f}")

    return fold_accuracies, mean_accuracy



Resposta à pergunta 6.1.4

Cada uma das abordagens de divisão de dados tem as suas vantagens e desvantagens, dependendo do contexto e do objetivo do modelo. A abordagem Train-Test (TT) divide o conjunto de dados em dois subconjuntos: um para treino e outro para teste. A principal vantagem desta abordagem é a sua simplicidade e rapidez, sendo adequada quando temos um grande conjunto de dados e a validação do modelo não precisa de ser altamente rigorosa. No entanto, a principal desvantagem é que a avaliação do modelo pode ser muito sensível à divisão específica dos dados, não representando bem a variabilidade do conjunto total de dados, o que pode levar a uma avaliação otimista ou pessimista.

A abordagem Train-Validation-Test (TVT), que utiliza uma divisão do treino em duas fases: treino e validação, permite uma avaliação mais robusta, pois o modelo é validado num conjunto de dados separado do treino, evitando o sobreajuste (overfitting). Contudo, esta abordagem é mais custosa em termos de tempo de computação, pois requer que o modelo seja treinado e validado em múltiplos subconjuntos. 

Por último, a K-Fold Cross-Validation é uma técnica que divide os dados em k subconjuntos (ou "folds") e treina e testa o modelo k vezes, cada vez com um fold diferente como conjunto de teste e os outros como conjunto de treino. A vantagem desta abordagem é que oferece uma avaliação mais robusta e menos sujeita a variações específicas de uma única divisão dos dados porque o modelo é treinado e testado em diferentes subconjuntos dos dados, permitindo que cada amostra do conjunto de dados seja utilizada tanto para treino quanto para teste. A desvantagem é o custo computacional, especialmente com grandes conjuntos de dados, pois o modelo precisa ser treinado k vezes. Em resumo, a escolha entre essas abordagens depende do tamanho do conjunto de dados, do tempo disponível para treino e do grau de robustez desejado na avaliação do modelo.

In [ ]:
#6.2 
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score

def evaluate_classification_performance(y_true, y_pred):

    conf_matrix = confusion_matrix(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='weighted')
    recall = recall_score(y_true, y_pred, average='weighted')
    f1 = f1_score(y_true, y_pred, average='weighted')

    return {
        "Confusion Matrix": conf_matrix,
        "Precision": precision,
        "Recall": recall,
        "F1-Score": f1,
    }



In [ ]:
#implementação KNN manual  
import numpy as np
from collections import Counter

class KNearestNeighbors:
    def __init__(self, k=3, weighted=False): #o self faz com que as variáveis possam ser acedidas por toda ainstância
       
        self.k = k
        self.weighted = weighted
        self.X_train = None
        self.Y_train = None

    def fit(self, X, Y):
      
        self.X_train = X
        self.Y_train = Y
        self.Y_train = np.array(Y).flatten()

    def _euclidean_distance(self, x1, x2):
      
        return np.sqrt(np.sum((x1 - x2) ** 2))

    def predict(self, X):
       
        predictions = []
        for x in X:
            # distância de x até todos os pontos de treino
            distances = [self._euclidean_distance(x, x_train) for x_train in self.X_train]
            
            # indices dos knn
            k_indices = np.argsort(distances)[:self.k]
            
            # labels dos knn
            k_labels = self.Y_train[k_indices]
            
            # voting
            if self.weighted:
                # Weighted voting - inverso da distância
                k_distances = np.array(distances)[k_indices]
                weights = 1 / (k_distances + 1e-5)  # evitar divisão por zero
                weighted_votes = {}
                for label, weight in zip(k_labels, weights):
                    weighted_votes[label] = weighted_votes.get(label, 0) + weight
                prediction = max(weighted_votes, key=weighted_votes.get)
            else:
                # Majority voting se weighted voting for false
                prediction = Counter(k_labels).most_common(1)[0][0]
            
            predictions.append(prediction)
        return predictions

    def accuracy(self, y_true, y_pred):
        
        correct = np.sum(np.array(y_true) == np.array(y_pred))
        total = len(y_true)
        accuracy = (correct / total) * 100
        print(accuracy)
        return accuracy
        


In [ ]:
#implementação KNN com a biblioteca

from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=10)
knn.fit(X_train, Y_train)
Y_val_pred = knn.predict(X_val)
accuracy = accuracy_score(Y_val, Y_val_pred)


In [ ]:
# Train-Validation-Test split: 40%-30%-30% com KNN

# normalizar
X_loaded_normalized = normalize_zscore(X_loaded)

# Split TVT
X_train, X_val, X_test, Y_train, Y_val, Y_test = nested_tvt_split(X_loaded_normalized, Y_loaded)

# PCA Function
def apply_pca(X_train, X_val, X_test, variance_threshold=0.97):
    transformed_train, components, _, _ = pca_with_explained_variance(X_train, variance_threshold)
    transformed_val = np.dot(X_val - np.mean(X_train, axis=0), components)
    transformed_test = np.dot(X_test - np.mean(X_train, axis=0), components)
    return transformed_train, transformed_val, transformed_test

# ReliefF Function
def apply_relieff(X_train, X_val, X_test, relief_scores, top_n=10):
    top_features = np.argsort(relief_scores)[-top_n:]
    X_train_reduced = X_train[:, top_features]
    X_val_reduced = X_val[:, top_features]
    X_test_reduced = X_test[:, top_features]
    return X_train_reduced, X_val_reduced, X_test_reduced

# KNN Cross-Validation Function
def cross_validate_knn(X_train, Y_train, k_values, X_val=None, Y_val=None):
    # Results dictionary
    results = {"k": [], "accuracy": [], "BIC": []}

    for k in k_values:
        # Train the KNN model
        knn = KNeighborsClassifier(n_neighbors=k, weights='distance')
        knn.fit(X_train, Y_train.ravel())
        
        # Predict on the validation set
        Y_val_pred = knn.predict(X_val)

        # Calculate accuracy
        accuracy = accuracy_score(Y_val, Y_val_pred)

        # Calculate BIC
        mse = np.mean((np.array(Y_val) - np.array(Y_val_pred)) ** 2)
        n = len(Y_val)
        bic = n * np.log(mse) + k * np.log(n) if mse > 0 else float('inf')  # Avoid log(0)

        # Store results
        results["k"].append(k)
        results["accuracy"].append(accuracy)
        results["BIC"].append(bic)

    return results

# Evaluate for All Features
k_values = [3, 5, 10, 15, 20]
results_all = cross_validate_knn(X_train, Y_train, k_values, X_val, Y_val)

# Evaluate for PCA Features
X_train_pca, X_val_pca, X_test_pca = apply_pca(X_train, X_val, X_test)
results_pca = cross_validate_knn(X_train_pca, Y_train, k_values, X_val_pca, Y_val)

# Evaluate for ReliefF Features
relief_scores = reliefF(X_train, Y_train)
X_train_relieff, X_val_relieff, X_test_relieff = apply_relieff(X_train, X_val, X_test, relief_scores)
results_relieff = cross_validate_knn(X_train_relieff, Y_train, k_values, X_val_relieff, Y_val)

# Print Results
print("All Features:")
for k, acc, bic in zip(results_all["k"], results_all["accuracy"], results_all["BIC"]):
    print(f"k={k}: Accuracy={acc:.4f}, BIC={bic:.2f}")

print("\nPCA Features:")
for k, acc, bic in zip(results_pca["k"], results_pca["accuracy"], results_pca["BIC"]):
    print(f"k={k}: Accuracy={acc:.4f}, BIC={bic:.2f}")

print("\nReliefF Features:")
for k, acc, bic in zip(results_relieff["k"], results_relieff["accuracy"], results_relieff["BIC"]):
    print(f"k={k}: Accuracy={acc:.4f}, BIC={bic:.2f}")



In [ ]:
# Train-Test split: 70%-30%; followed by 5-Fold Cross Validation on the training set com KNN

from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier

# Train-Test Split (70%-30%)
X_train, X_test, Y_train, Y_test = train_test_split(X_loaded_normalized, Y_loaded, test_size=0.3, random_state=42)

# KNN Cross-Validation Function
def cross_validate_knn(X_train, Y_train, k_values, n_splits=5):
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    
    # resultados
    results = {"k": [], "accuracy": [], "BIC": []}

    for k in k_values:
        fold_accuracies = []
        fold_bics = []
        
        for train_idx, val_idx in kf.split(X_train):
            X_train_fold, X_val_fold = X_train[train_idx], X_train[val_idx]
            Y_train_fold, Y_val_fold = Y_train[train_idx], Y_train[val_idx]

            # Treinar e predict com KNN 
            knn = KNeighborsClassifier(n_neighbors=k, weights='distance')
            knn.fit(X_train_fold, Y_train_fold)
            Y_val_pred = knn.predict(X_val_fold)

            # calcular accuracy
            accuracy = accuracy_score(Y_val_fold, Y_val_pred)
            fold_accuracies.append(accuracy)

            # Calcular BIC
            mse = np.mean((np.array(Y_val_fold) - np.array(Y_val_pred)) ** 2)
            bic = len(Y_val_fold) * np.log(mse) + k * np.log(len(Y_val_fold)) if mse > 0 else float('inf')
            fold_bics.append(bic)

        # resultados médios por fold 
        mean_accuracy = np.mean(fold_accuracies)
        mean_bic = np.mean(fold_bics)

        results["k"].append(k)
        results["accuracy"].append(mean_accuracy)
        results["BIC"].append(mean_bic)

    return results

# PCA 
def apply_pca(X_train, variance_threshold=0.99):
    transformed_train, components, _, _ = pca_with_explained_variance(X_train, variance_threshold)
    return transformed_train, components

# ReliefF 
def apply_relieff(X_train, relief_scores, top_n=10):
    top_features = np.argsort(relief_scores)[-top_n:]
    return X_train[:, top_features]

# All Features
k_values = [3, 5, 10, 15, 20]
results_all = cross_validate_knn(X_train, Y_train.ravel(), k_values)

# PCA Features
X_train_pca, _ = apply_pca(X_train)
results_pca = cross_validate_knn(X_train_pca, Y_train.ravel(), k_values)

# ReliefF Features
relief_scores = reliefF(X_train, Y_train)
X_train_relieff = apply_relieff(X_train, relief_scores)
results_relieff = cross_validate_knn(X_train_relieff, Y_train.ravel(), k_values)

# Resultados
print("All Features:")
for k, acc, bic in zip(results_all["k"], results_all["accuracy"], results_all["BIC"]):
    print(f"k={k}: Accuracy={acc:.4f}, BIC={bic:.2f}")

print("\nPCA Features:")
for k, acc, bic in zip(results_pca["k"], results_pca["accuracy"], results_pca["BIC"]):
    print(f"k={k}: Accuracy={acc:.4f}, BIC={bic:.2f}")

print("\nReliefF Features:")
for k, acc, bic in zip(results_relieff["k"], results_relieff["accuracy"], results_relieff["BIC"]):
    print(f"k={k}: Accuracy={acc:.4f}, BIC={bic:.2f}")





In [ ]:
# Train-Test split: 70%-30%; followed by 5-Fold Cross Validation on the training set com Random Forest

from sklearn.ensemble import RandomForestClassifier

# Train-Test Split (70%-30%)
X_train, X_test, Y_train, Y_test = train_test_split(X_loaded_normalized, Y_loaded, test_size=0.3, random_state=42)

# KNN Cross-Validation Function
def cross_validate_rf(X_train, Y_train, n_estimators_values, n_splits=5):
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    
    # resultados
    results = {"n_estimators": [], "accuracy": [], "BIC": []}

    for n_estimators in n_estimators_values:
        fold_accuracies = []
        fold_bics = []
        
        for train_idx, val_idx in kf.split(X_train):
            X_train_fold, X_val_fold = X_train[train_idx], X_train[val_idx]
            Y_train_fold, Y_val_fold = Y_train[train_idx], Y_train[val_idx]

            # Treinar e predict com Random Forest
            rf = RandomForestClassifier(n_estimators=n_estimators, random_state=42)
            rf.fit(X_train_fold, Y_train_fold.ravel())
            Y_val_pred = rf.predict(X_val_fold)

            # Calcular accuracy
            accuracy = accuracy_score(Y_val_fold, Y_val_pred)
            fold_accuracies.append(accuracy)

            # Calcular BIC
            mse = np.mean((np.array(Y_val_fold) - np.array(Y_val_pred)) ** 2)
            bic = len(Y_val_fold) * np.log(mse) + n_estimators * np.log(len(Y_val_fold)) if mse > 0 else float('inf')
            fold_bics.append(bic)

        # resultados médios por folds
        mean_accuracy = np.mean(fold_accuracies)
        mean_bic = np.mean(fold_bics)

        results["n_estimators"].append(n_estimators)
        results["accuracy"].append(mean_accuracy)
        results["BIC"].append(mean_bic)

    return results

# PCA 
def apply_pca(X_train, variance_threshold=0.97):
    transformed_train, components, _, _ = pca_with_explained_variance(X_train, variance_threshold)
    return transformed_train, components

# ReliefF 
def apply_relieff(X_train, relief_scores, top_n=10):
    top_features = np.argsort(relief_scores)[-top_n:]
    return X_train[:, top_features]

# All Features
n_estimators_values = [10, 50, 100, 200, 500]
results_all = cross_validate_rf(X_train, Y_train, n_estimators_values)

# PCA Features
X_train_pca, _ = apply_pca(X_train)
results_pca = cross_validate_rf(X_train_pca, Y_train, n_estimators_values)

# ReliefF Features
relief_scores = reliefF(X_train, Y_train.ravel())
X_train_relieff = apply_relieff(X_train, relief_scores)
results_relieff = cross_validate_rf(X_train_relieff, Y_train, n_estimators_values)

# Resultados
print("All Features:")
for n_est, acc, bic in zip(results_all["n_estimators"], results_all["accuracy"], results_all["BIC"]):
    print(f"n_estimators={n_est}: Accuracy={acc:.4f}, BIC={bic:.2f}")

print("\nPCA Features:")
for n_est, acc, bic in zip(results_pca["n_estimators"], results_pca["accuracy"], results_pca["BIC"]):
    print(f"n_estimators={n_est}: Accuracy={acc:.4f}, BIC={bic:.2f}")

print("\nReliefF Features:")
for n_est, acc, bic in zip(results_relieff["n_estimators"], results_relieff["accuracy"], results_relieff["BIC"]):
    print(f"n_estimators={n_est}: Accuracy={acc:.4f}, BIC={bic:.2f}")

In [ ]:
# Selecionei o modelo com melhor accuraccy - Random Forest Classifier com n_estimators=500 com todas as features
rf_classifier = RandomForestClassifier(n_estimators=500, random_state=42)
rf_classifier.fit(X_train, Y_train.ravel())

# Guardar o melhor modelo com pickle
with open('random_forest_wheelchair_game.pkl', 'wb') as file:
    pickle.dump(rf_classifier, file)

print("Model saved successfully!")
